# Week 08 Problem Set

## Homework

In [1]:
%load_ext nb_mypy
%nb_mypy On

ModuleNotFoundError: No module named 'nb_mypy'

In [2]:
from typing import TypeAlias
from typing import Optional, Any, Callable, Iterator, Iterable, cast
from __future__ import annotations

Number: TypeAlias = int | float

**HW1.** *Comments:* Write a state machine whose inputs are the characters of a string. The string contains the code for a computer program. The output of the state machine are either:
- the input character if it is part of a comment, or
- `None`, otherwise.

Comment starts with a `#` character and continue to the end of the current line. If you want to create a string that contains a new line character, you can use `\n`.

For example,
```
inpstr = "def func(x): # comment\n    return 1"
m = CommentsSM()
print(m.transduce(inpstr))
```

The expected output is:
```
[None, None, None, None, None, None, None, None, None, None, None, None, None, "#", " ", "c", "o", "m", "m", "e", "n", "t", None, None, None, None, None, None, None, None, None, None, None, None, None]
```

You should start by drawing a state transition diagram indicating the states and what inputs cause transition to which other states. Use the test case above to determine if your state transition diagram is correct. You should begin writing your program only when you are confident that your diagram is correct. 

In [31]:
# Copy over the implementation of StateMachine from Cohort
from abc import ABC, abstractmethod

class StateMachine(ABC):
    def start(self):
        self.state = self.start_state
    
    def step(self, inp):
        prev = self.state
        cur_state, output = self.get_next_values(prev,inp)
        self.state = cur_state
        return output
        
    def transduce(self, inp_list):
        self.start()
        ret = []
        for item in inp_list:
            if (self.is_done()):
                return ret
            ret.append(self.step(item))
        print(ret)
        return ret
        

    @property
    @abstractmethod
    def start_state(self):
        pass 

    @abstractmethod
    def get_next_values(self, state, inp):
        pass

    def done(self, state):
        return False

    def is_done(self):
        return self.done(self.state)


In [51]:
class CommentsSM(StateMachine):
    
    @property
    def start_state(self) -> str:
        return 0

    def get_next_values(self, state: str, inp: str) -> tuple[str, Optional[str]]:
        next_state, output = state, None
        item = inp
        op = None
        if (state == 0):
            if (item == "#"):
                next_state = 1
                op = item
        else:
            if (item == '\n'):
                next_state = 0
            else:
                op = item
            
        output = op
        return next_state, output

In [52]:
inpstr: str = "def func(x): # comment\n    return 1"
m: CommentsSM = CommentsSM()
out: list[Optional[str]] = m.transduce(inpstr)
assert out == [None, None, None, None, None, None, None, None, None, None, None, None, None, "#", " ", "c", "o", "m", "m", "e", "n", "t", None, None, None, None, None, None, None, None, None, None, None, None, None]

###
### AUTOGRADER TEST - DO NOT REMOVE
###


[None, None, None, None, None, None, None, None, None, None, None, None, None, '#', ' ', 'c', 'o', 'm', 'm', 'e', 'n', 't', None, None, None, None, None, None, None, None, None, None, None, None, None]


In [53]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###

###
### AUTOGRADER TEST - DO NOT REMOVE
###


**HW2.** *First Word:* Write a state machine whose inputs are the characters of a string and which outputs either:
- the input character if it is part of the first word on a line, or
- `None`, otherwise

For the purposes here, a word is any sequence of consecutive characters that does not contain spaces or end-of-line characters. In this problem, comments have no special status. This means that if the line begins with `# `, then the first word is `#`. 

For example, 
```
inpstr = "def func(x): # comment\n    return 1"
m = FirstWordSM()
print(m.transduce( inpstr))
```

The expected output is:
```
["d", "e", "f", None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, "r", "e", "t", "u", "r", "n", None, None]
```

In [95]:
class FirstWordSM(StateMachine):
  
    @property
    def start_state(self) -> str:
        return -1
  
    def get_next_values(self, state: str, inp: str) -> tuple[str, Optional[str]]:
        next_state, output = state, None
        if (state == 1):
            if (inp == " "):
                next_state = 0
            elif (inp == "\n"):
                next_state = -1
                output = None
            else:
                output = inp
        elif (state == -1):
            if (inp != " " and inp != "\n"):
                next_state = 1
                output = inp
        else:
            if (inp == "\n"):
                next_state = -1
        #print(inp, next_state, output)
        return next_state, output

In [96]:
inpstr: str = "# \n#wowio\n 8 9"
m: FirstWordSM = FirstWordSM()
out: list[Optional[str]] = m.transduce(inpstr)

['#', None, None, '#', 'w', 'o', 'w', 'i', 'o', None, None, '8', None, None]


In [76]:
inpstr: str = "def func(x): # comment\n    return 1"
m: FirstWordSM = FirstWordSM()
out: list[Optional[str]] = m.transduce(inpstr)
assert out == ["d", "e", "f", None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, "r", "e", "t", "u", "r", "n", None, None]


###
### AUTOGRADER TEST - DO NOT REMOVE
###


['d', 'e', 'f', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 'r', 'e', 't', 'u', 'r', 'n', None, None]


In [61]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###

###
### AUTOGRADER TEST - DO NOT REMOVE
###


**HW3.** *Robot:* Write a State Machine class that represent a robot. The dimension of the world and the robot initial position should be specified during the class instantiation. The robot can take in the following input:
- "left"
- "right"
- "up"
- "down"

The initial position of the robot is specified during the object instantiation and the input should modify the position of the robot. The robot position must not change if it exceed the boundary. At each step, the robot should output the updated position. 

In [62]:
class Position:
  
    def __init__(self, x: int=0, y: int=0) -> None:
        self.x: int = x
        self.y: int = y
  
    def __str__(self) -> str:
        return f"({self.x:}, {self.y:})"

class Dimension:

    def __init__(self, width: int=0, height: int=0) -> None:
        self.width: int = width
        self.height: int = height
  
    def __str__(self) -> str:
        return f"width: {self.width:}, height: {self.height:}"

In [66]:
class RobotSM(StateMachine):
    @property
    def start_state(self) -> Position:
        return self.__start_state
    
    @start_state.setter
    def start_state(self, value: Position) -> None:
        self.__start_state: Position = value

    def __init__(self, init_pos: Position, dimension: Dimension) -> None:
        self.world_dim: Dimension = dimension
        self.start_state = init_pos
  
    def get_next_values(self, state: Position, inp: str) -> tuple[Position, Position]:
        next_state = state
        if (inp == "left"):
            next_state.x -= 1
        elif (inp == "right"):
            next_state.x += 1
        elif (inp == "up"):
            next_state.y += 1
        else:
            next_state.y -= 1

        next_state.y = min(self.world_dim.height-1, max(0, next_state.y))
        next_state.x = min(self.world_dim.width-1, max(0, next_state.x))
        
        return next_state, next_state

In [67]:
robot: RobotSM = RobotSM(Position(0, 0), Dimension(5, 5))
robot.start()
robot.transduce(["right", "right", "up", "up", "up", "left", "down"])
pos: Position = robot.state
assert pos.x == 1 and pos.y == 2

###
### AUTOGRADER TEST - DO NOT REMOVE
###


1 0
2 0
2 1
2 2
2 3
1 3
1 2
[<__main__.Position object at 0x10a143380>, <__main__.Position object at 0x10a143380>, <__main__.Position object at 0x10a143380>, <__main__.Position object at 0x10a143380>, <__main__.Position object at 0x10a143380>, <__main__.Position object at 0x10a143380>, <__main__.Position object at 0x10a143380>]


In [15]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###

###
### AUTOGRADER TEST - DO NOT REMOVE
###


**HW4.** *Search SM:* Write a function `sm_search` that takes in the following arguments:
- `sm_to_search`: is the State Machine instance to search. This argument is of the type `MapSM` as defined in CS4. You should use the `get_next_values()` of this State Machine instance to explore the next state in your search.
- `initial_state`: is the start state of the search. If it is not provided, it should be assigned to the `start_state` of sm_search. 
- `goal_test`: is a function that returns `True` if the argument is the end state of the search. If it is not provided, it should be eassigned to the `done` function of the state machine.

This function performs a **breadth-first-search** algorithm to explore the next states. 

The output is a `list` of `Step` instances from the `init_state` to the end state which is determined by the `goal_test` function.

This problem requires you to complete the following:
- `Queue` class from Problem Set 4 HW2.
- `MapSM` class in CS4.
- `SearchNode` and `Step` classes in CS5.

In [68]:
class Stack:
    def __init__(self) -> None:
        self.__items: list[Any] = []
        
    def push(self, item: Any):
        self.__items.append(item)

    def pop(self) -> Any:
        if (len(self.__items) == 0):
            return None
        return self.__items.pop()

    def peek(self) -> Any:
        return self.__items[-1]

    @property
    def is_empty(self) -> bool:
        return (len(self.__items) == 0)

    @property
    def size(self):
        return len(self.__items)



class Queue:
    def __init__(self) -> None:
        self.left_stack: Stack = Stack()
        self.right_stack: Stack = Stack()
        
    def enqueue(self, item: Any):
        self.left_stack.push(item)

    def dequeue(self) -> Any:
        if (self.right_stack.is_empty):
            while(self.left_stack.size > 0):
                self.right_stack.push(self.left_stack.pop())
        return self.right_stack.pop()
            

    def peek(self) -> Any:
        if (self.right_stack.size + self.left_stack.size == 0):
            return None
        if (self.right_stack.size > 0):
            return self.right_stack.peek()
        while(self.left_stack.size > 0):
            self.right_stack.push(self.left_stack.pop())
        return self.right_stack.peek()

    @property
    def is_empty(self) -> bool:
        return (self.left_stack.size + self.right_stack.size == 0)

    @property
    def size(self):
        return self.left_stack.size + self.right_stack.size


In [69]:
from abc import abstractmethod

class StateSpaceSearch(StateMachine):
    @property
    @abstractmethod
    def statemap(self):
        pass

    @property
    @abstractmethod
    def legal_inputs(self):
        pass

    @property
    def start_state(self):
        return self.__start_state
    
    @start_state.setter
    def start_state(self, value):
        self.__start_state = value

In [70]:
class MapSM(StateSpaceSearch):
        
    def __init__(self, start: str) -> None:
        self.start_state = start

    @property
    def statemap(self) -> dict[str, list[str]]:
        statemap = {"S": ["A", "B"],
                    "A": ["S", "C", "D"],
                    "B": ["S", "D", "E"],
                    "C": ["A", "F"],
                    "D": ["A", "B", "F", "H"],
                    "E": ["B", "H"],
                    "F": ["C", "D", "G"],
                    "H": ["D", "E", "G"],
                    "G": ["F", "H"]}
        return statemap

    @property
    def legal_inputs(self) -> set[int]:
        n = 0
        for key, value in self.statemap.items():
            n = max(n, len(value))
        
        return set(range(0,n))

  
    def get_next_values(self, state: str, inp: int) -> tuple[str, str]:
        neigh = self.statemap[state]
        next_state = state
        if (len(neigh) > inp):
            next_state = neigh[inp]
        return next_state, next_state
            
        
            
    

In [71]:
# Copy over the implementations of Step and SearchNode from Cohort
class Step:
    def __init__(self, action: Optional[int], state: str) -> None:
        self.action: Optional[int] = action
        self.state: str = state
    
    def __eq__(self, other) -> bool:
        return self.action == other.action and self.state == other.state
  
    def __str__(self) -> str:
        return f"action: {self.action:}, state: {self.state:}"

class SearchNode:
    def __init__(self, action: Optional[int], state: str, parent: Optional[SearchNode]) -> None:
        self.state: str = state
        self.action: Optional[int] = action
        self.parent: Optional[SearchNode] = parent
  
    def path(self) -> list[Step]:
        if (self.parent == None):
            return [Step(self.action, self.state)]
        return self.parent.path() + [Step(self.action, self.state)]
  
    def in_path(self, state: str) -> bool:
        if self.state == state:
            return True
        elif self.parent is None:
            return False
        else:
            return self.parent.in_path(state)
  
    def __eq__(self, other) -> bool:
        if self is None and other is None:
            return True
        elif self is None:
            return False
        elif other is None:
            return False
        else:
            return self.state == other.state and self.parent == other.parent and \
                   self.action == other.action


In [72]:
def sm_search(sm_to_search: StateSpaceSearch, initial_state: Optional[str]=None, goal_test: Optional[Callable]=None) -> Optional[list[Step]]:
    # check if initial_state is provided
    # if it is, use it
    # otherwise, get the start state of sm_to_search
    if initial_state is None:
        # replace None to take the start state of sm_to_search
        init_state: str = sm_to_search.start_state
    else:
        init_state = initial_state
  
    # check if goal_test is provided
    # if it is, use it
    # otherwise, use the done method as the goal function
    # taken from sm_to_search
    if goal_test is None:
        goal_func: Callable = sm_to_search.done
    else:
        goal_func = goal_test
  
    # create a Queue instance to store the node to explore
    # replace the None below
    agenda = Queue()
    ###
    ### YOUR CODE HERE
    ###
  
    # if the initial state is the goal state, 
    # then we are done and exit
    if goal_func(init_state):
        return [Step(None, init_state)]
  
    # otherwise, add the current node into the agenda 
    agenda.enqueue(SearchNode(None, init_state, None))
    
    # explore as long as the Queue is not empty
    while not agenda.is_empty:
        
        # replace None to take out the parent from the Queue
        parent = agenda.dequeue()
        
        # create a list to keep track which child state have been explored
        new_child_state: list[str] = []
        
        # get all the legal input values
        actions: set[int] = sm_to_search.legal_inputs
        
        #iterate over all legal inputs
        for a in actions:
            # get the next possible state using the current action
            # call get_next_values to get the next state
            # replace the None below
            new_s = sm_to_search.get_next_values(parent.state, a)[0]
            
            # create a new search node from the new_s
            # replace the None below
            new_n = SearchNode(a, new_s, parent)

            
            # if the new state is the goal state, then we exit and return the path
            if goal_func(new_s):
                return new_n.path()
            
            # if the new state is already in the list of new child state, ignore it
            elif new_s in new_child_state:
                pass
            
            # if the new state is in the path of the current node, ignore it
            elif parent.in_path(new_s):
                pass
            
            # otherwise, add the new state into the list
            # and the new node into the Queue
            else:
                # step 1. add the new state into the new_child_state
                new_child_state.append(new_s)
                
                # step 2. add the new node into the Queue
                agenda.enqueue(new_n)
                pass
    return None

In [73]:
mapSM: MapSM = MapSM("S")
ans: Optional[list[Step]] = sm_search(mapSM , "S" , lambda s: s=="H" )
assert ans is not None
steps: list[tuple[Optional[int], str]] = [(step.action, step.state) for step in ans]
assert steps == [(None, "S"), (0, "A"), (2, "D"), (3, "H")]
for step in ans:
    print(step)

###
### AUTOGRADER TEST - DO NOT REMOVE
###


action: None, state: S
action: 0, state: A
action: 2, state: D
action: 3, state: H


In [74]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###
###
### AUTOGRADER TEST - DO NOT REMOVE
###
